In [4]:
import time

In [5]:
import pandas as pd
import sqlalchemy
from binance.client import Client
from binance import BinanceSocketManager

In [9]:
with open("api.txt", 'r') as file:
    api_key = file.readline().replace('\n','')
    api_secret = file.readline()

api_key

'mJRjVZ1KVpsSZOpcrabUnXLFd5vryGNEqlxh7JwOkXNgrhzL1fxBuZXl4UeosZEP'

In [11]:
client = Client(api_key, api_secret)

In [12]:
bsm = BinanceSocketManager(client)

In [13]:
socket = bsm.trade_socket('BTCUSDT')

In [17]:
n_max = 0
while n_max < 100:
    start_time = time.time()
    await socket.__aenter__()
    msg = await socket.recv()
    df = createframe(msg)
    df.to_sql('BTCUSDT', engine, if_exists='append', index = False)
    print(df)
    n_max += 1

    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.247  19190.53
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.250  19190.54
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.257  19190.48
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.257  19190.48
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.295  19190.91
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.295  19190.96
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.295  19190.99
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.331  19190.98
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.349  19190.49
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.597  19190.48
    symbol                    Time     Price
0  BTCUSDT 2022-10-15 04:55:59.658  19190.49
    symbol

In [15]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:, ['s', 'E', 'p']]
    df.columns = ['symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [14]:
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream1st.db')